# FLAN-T5 Finetuning for QA on Yahoo Data

## Import the libraries

In [1]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
import warnings
from transformers import logging

logging.set_verbosity_info()
warnings.filterwarnings('ignore')

## Data Loading

In [3]:
from datasets import load_dataset

In [4]:
# Acquire the training data from Hugging Face
#full_dataset = load_dataset("infinite-dataset-hub/Linux_Command_Classification")
full_dataset = load_dataset("rohanbalkondekar/linux_commands")

In [5]:
# Check the length of the data and its structure
full_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'Unnamed: 3', 'Unnamed: 4'],
        num_rows: 150
    })
})

In [6]:
SAMPLE_SIZE = 100
subset = full_dataset['train'].select(range(SAMPLE_SIZE))

In [7]:
subset = subset.train_test_split(test_size=0.3)

In [8]:
subset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'Unnamed: 3', 'Unnamed: 4'],
        num_rows: 70
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'Unnamed: 3', 'Unnamed: 4'],
        num_rows: 30
    })
})

In [9]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME,legacy=False,clean_up_tokenization_spaces=False)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

loading file spiece.model from cache at /home/shaima0d/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/spiece.model
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/shaima0d/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/shaima0d/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/tokenizer_config.json
loading file tokenizer.json from cache at /home/shaima0d/.cache/huggingface/hub/models--google--flan-t5-base/snapshots/7bcac572ce56db69c1ea7c8af255c5d7c9672fc2/tokenizer.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file config.json from cache at /home/shaima0d/.cache/huggingface/hub/models--googl

In [10]:
# We prefix our tasks with "answer the question"
prefix = "Explain the linux command: "

# Define the preprocessing function

def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["input"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["output"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
# Map the preprocessing function across our dataset
tokenized_dataset = subset.map(preprocess_function, batched=True)

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [12]:
# Set up Rouge score for evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [13]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return result

In [14]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 1
PER_DEVICE_EVAL_BATCH = 1
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 2
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=L_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    push_to_hub=False
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [15]:
# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [16]:
print(f'Number of training itreations: {NUM_EPOCHS*len(subset["train"])/BATCH_SIZE}')

Number of training itreations: 210.0


In [17]:
# Trigger the model training
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: output, Unnamed: 4, input, Unnamed: 3, instruction. If output, Unnamed: 4, input, Unnamed: 3, instruction are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.


[2024-12-12 16:45:03,085] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.


/ibex/user/shaima0d/KSL_Trainings/rts-tutorials/LLM-finetuning/install/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
***** Running training *****
  Num examples = 70
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 210
  Number of trainable parameters = 247,577,856


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,3.158439,0.226774,0.062947,0.212058,0.213172
2,No log,3.337084,0.282485,0.107742,0.266973,0.265238
3,No log,3.710948,0.284263,0.100643,0.254651,0.257598


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: output, Unnamed: 4, input, Unnamed: 3, instruction. If output, Unnamed: 4, input, Unnamed: 3, instruction are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 30
  Batch size = 1
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: output, Unnamed: 4, input, Unnamed: 3, instruction. If output, Unnamed: 4, input, Unnamed: 3, instruction are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 30
  Batch size = 1
Saving model checkpoint to ./results/checkpoint-210
Configuration saved in ./results/checkpoint-210/config.json
Configuration saved in ./results/checkpoint-210/generation_con

TrainOutput(global_step=210, training_loss=2.368891398111979, metrics={'train_runtime': 43.53, 'train_samples_per_second': 4.824, 'train_steps_per_second': 4.824, 'total_flos': 3205787839488.0, 'train_loss': 2.368891398111979, 'epoch': 3.0})

## Run in inference mode

In [18]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

#last_checkpoint = "./results/checkpoint-22500"
last_checkpoint = "./results/checkpoint-21"
finetuned_model = T5ForConditionalGeneration.from_pretrained(last_checkpoint)
tokenizer = T5Tokenizer.from_pretrained(last_checkpoint)

loading configuration file ./results/checkpoint-21/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: 

In [19]:
my_question = "ls"
inputs = "Explain the linux command: " + my_question

inputs = tokenizer(inputs, return_tensors="pt")

In [20]:
outputs = finetuned_model.generate(**inputs)
answer = tokenizer.decode(outputs[0])

print("Answer:", answer)

Answer: <pad> Command used to create directories in Linux</s>
